In [119]:
from bs4 import BeautifulSoup
with open('old1.xml', 'r') as f: 
    data = f.read()
Bs_data = BeautifulSoup(data, "xml")

from xml.dom import minidom
mydoc = minidom.parse('old1.xml')
managedObject = mydoc.getElementsByTagName('managedObject')
version_site = managedObject[0].attributes['version'].value
Nokia_site_name = managedObject[0].attributes['distName'].value

site_name_data = Bs_data.find('managedObject', {'distName':Nokia_site_name})

import re
r1=re.findall(r'<p name="(.*?)</p>',str(site_name_data.find_all('p')))

import pandas as pd
df = pd.DataFrame(r1,columns=['data'])

df[['para','val']] = df.data.str.split('">',expand=True)

df['Parameter']=df['para'].str.replace("'", '').str.replace("]", '')
df['Value']=df['val'].str.replace("'", '').str.replace("]", '')

Site_name = df['Value'][4]

data = {
        'distName':[Nokia_site_name],
        'Parameter':['Version'],
        'Value':[version_site]}

Main_DF=pd.DataFrame(data)


In [120]:
for Object in range(0,managedObject.length):
    name = managedObject[Object].attributes['distName'].value
    site_data = Bs_data.find('managedObject', {'distName':name})
    r1=re.findall(r'<p name="(.*?)</p>',str(site_data.find_all('p')))
    dft = pd.DataFrame(r1,columns=['data'])
    dft[['para','val']] = dft.data.str.split('">',expand=True)
    dft['distName'] = name
    dft['Parameter']=dft['para'].str.replace("'", '').str.replace("]", '')
    dft['Value']=dft['val'].str.replace("'", '').str.replace("]", '')
    final_temp_DF = dft.drop(['para', 'val','data'], axis=1)
    Main_DF=Main_DF.append(final_temp_DF,ignore_index=True)

In [130]:
Main_DF['distName']=Main_DF['distName'].str.replace(Nokia_site_name,Site_name)
Main_DF.to_csv(r'{}.csv'.format(Site_name), index = False)